In [3]:
import pandas as pd

In [4]:
# Bring each CSV into a separate data frame
accident_df = pd.read_csv("resources\ACCIDENT.csv")
accident_df

C:\Users\stuar\.conda\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ACCIDENT_NO,ACCIDENTDATE,ACCIDENTTIME,ACCIDENT_TYPE,Accident Type Desc,DAY_OF_WEEK,Day Week Description,DCA_CODE,DCA Description,DIRECTORY,...,NO_PERSONS,NO_PERSONS_INJ_2,NO_PERSONS_INJ_3,NO_PERSONS_KILLED,NO_PERSONS_NOT_INJ,POLICE_ATTEND,ROAD_GEOMETRY,Road Geometry Desc,SEVERITY,SPEED_ZONE
0,T20060000010,13/01/2006,12:42:00,1,Collision with vehicle,6,Friday,113,RIGHT NEAR (INTERSECTIONS ONLY),MEL,...,6,0,1,0,5,1,1,Cross intersection,3,60
1,T20060000018,13/01/2006,19:10:00,1,Collision with vehicle,6,Friday,113,RIGHT NEAR (INTERSECTIONS ONLY),MEL,...,4,0,1,0,3,1,2,T intersection,3,70
2,T20060000022,14/01/2006,12:10:00,7,Fall from or in moving vehicle,7,Saturday,190,FELL IN/FROM VEHICLE,MEL,...,2,1,0,0,1,1,5,Not at intersection,2,100
3,T20060000023,14/01/2006,11:49:00,1,Collision with vehicle,7,Saturday,130,REAR END(VEHICLES IN SAME LANE),MEL,...,2,1,0,0,1,1,2,T intersection,2,80
4,T20060000026,14/01/2006,10:45:00,1,Collision with vehicle,7,Saturday,121,RIGHT THROUGH,MEL,...,3,0,3,0,0,1,5,Not at intersection,3,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203703,T20200019239,1/11/2020,12:11:00,1,Collision with vehicle,0,Sunday,142,LEAVING PARKING,MEL,...,4,1,0,0,3,1,5,Not at intersection,2,50
203704,T20200019247,1/11/2020,15:30:00,4,Collision with a fixed object,1,Sunday,171,LEFT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICL...,MEL,...,2,2,0,0,0,1,5,Not at intersection,2,999
203705,T20200019250,1/11/2020,18:00:00,1,Collision with vehicle,0,Sunday,116,LEFT NEAR (INTERSECTIONS ONLY),MEL,...,2,1,0,0,1,1,1,Cross intersection,2,60
203706,T20200019253,1/11/2020,12:00:00,6,Vehicle overturned (no collision),1,Sunday,180,OFF CARRIAGEWAY ON RIGHT BEND,VCD,...,1,1,0,0,0,1,5,Not at intersection,2,80


In [6]:
# Bring each CSV into a separate data frame
accident_event_df = pd.read_csv("resources\ACCIDENT_EVENT.csv")
accident_event_df

,ACCIDENT_NO,EVENT_SEQ_NO,EVENT_TYPE,Event Type Desc,VEHICLE_1_ID,VEHICLE_1_COLL_PT,Vehicle 1 Coll Pt Desc,VEHICLE_2_ID,VEHICLE_2_COLL_PT,Vehicle 2 Coll Pt Desc,PERSON_ID,OBJECT_TYPE,Object Type Desc
0,T20060000010,1.0,C,Collision,B,2,Right side (forwards),A,F,Front,,99.0,Not Applicable
1,T20060000018,1.0,C,Collision,B,F,Front,A,9,Not known or Not Applicable,,99.0,Not Applicable
2,T20060000022,1.0,2,Fell from vehicle,A,9,Not known or Not Applicable,,,,01,99.0,Not Applicable
3,T20060000023,1.0,C,Collision,A,F,Front,B,R,Rear,,99.0,Not Applicable
4,T20060000026,1.0,C,Collision,A,9,Not known or Not Applicable,B,9,Not known or Not Applicable,,99.0,Not Applicable
...,...,...,...,...,...,...,...,...,...,...,...,...,...
326633,T20200019250,1.0,C,Collision,A,1,Right front corner,B,5,Left front corner,,99.0,Not Applicable
326634,T20200019253,1.0,1,Rollover on/off carriageway,A,5,Left front corner,,,,,99.0,Not Applicable
326635,T20200019253,2.0,2,Fell from vehicle,A,9,Not known or Not Applicable,,,,,99.0,Not Applicable
326636,T20200019417,1.0,3,Ran off carriageway,A,9,Not known or Not Applicable,,,,,99.0,Not Applicable


In [7]:
# Bring each CSV into a separate data frame
accident_location_df = pd.read_csv("resources\ACCIDENT_LOCATION.csv")
accident_location_df

,ACCIDENT_NO,NODE_ID,ROAD_ROUTE_1,ROAD_NAME,ROAD_TYPE,ROAD_NAME_INT,ROAD_TYPE_INT,DISTANCE_LOCATION,DIRECTION_LOCATION,NEAREST_KM_POST,OFF_ROAD_LOCATION
0,T20060000010,43078,2090.0,FOSTER,STREET,MCCRAE,STREET,0.0,SW,NaN,NaN
1,T20060000018,29720,5057.0,HALLAM,ROAD,BELGRAVE-HALLAM,ROAD,70.0,S,NaN,NaN
2,T20060000022,203074,9999.0,BROWNS,ROAD,TRUEMANS,ROAD,210.0,W,NaN,NaN
3,T20060000023,55462,2400.0,SPRINGVALE,ROAD,KEYSBOROUGH,AVENUE,0.0,N,NaN,NaN
4,T20060000026,202988,9999.0,ELIZABETH,AVENUE,GREENHOOD,CRESCENT,20.0,N,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
203703,T20200019239,665835,9999.0,ALEXANDRA,AVENUE,ANDERSON,STREET,147.0,W,NaN,NaN
203704,T20200019247,665912,9999.0,MATTHEWS,ROAD,FITZGERALD,ROAD,413.0,S,NaN,NaN
203705,T20200019250,226603,5814.0,PLENTY,ROAD,BUSH,BOULEVARD,0.0,SW,NaN,NaN
203706,T20200019253,665836,2560.0,OMEO,HIGHWAY,CALLAGHANS,ROAD,4531.0,NE,NaN,NaN
